<a href="https://colab.research.google.com/github/Varfalamei/cross-domain-recsys/blob/lab3/notebooks/emb_from_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!gdown --id 1NM4uSjSg0e2LPBT_aC0v1XaRRh932jxJ
!gdown --id 19IDNSFnAh-JiVx05BSJJRmU-eTdzlG1T

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1NM4uSjSg0e2LPBT_aC0v1XaRRh932jxJ
To: /content/books.csv
100% 69.0M/69.0M [00:00<00:00, 209MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=19IDNSFnAh-JiVx05BSJJRmU-eTdzlG1T
To: /content/films.csv
100% 28.6M/28.6M [00:00<00:00, 117MB/s] 


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 14.8 MB/s 
     |████████████████████████████████| 7.6 MB 52.9 MB/s 
     |████████████████████████████████| 163 kB 65.4 MB/s 


In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoTokenizer, AutoModel
from collections import defaultdict

In [27]:
books = pd.read_csv('books.csv')
films = pd.read_csv('films.csv')

In [28]:
books = books[~books.isna().any(axis=1)].reset_index(drop=True)
films = films[~films.isna().any(axis=1)].reset_index(drop=True)

In [29]:
# list_of_idx_books = [
#     99, 1980, 10618, 18685, 33857, 54081, 17357, 659, 35770
# ]

# books = books[books.id.isin(list_of_idx_books)]

In [30]:
films.shape

(23120, 7)

In [31]:
books.shape

(57588, 6)

In [14]:
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')
model = AutoModel.from_pretrained('cointegrated/rubert-tiny2')

Downloading:   0%|          | 0.00/401 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [16]:
embed_bert_cls("привет как дела", model, tokenizer).shape

(312,)

In [33]:
books_small = books.copy()
films_small = films.copy()

In [34]:
films_small.shape

(23120, 7)

In [35]:
books_small_emb = books_small.annotation.apply(lambda x: embed_bert_cls(x, model, tokenizer))
films_small_emb = films_small.description.apply(lambda x: embed_bert_cls(x, model, tokenizer))

In [36]:
books_small_emb = np.stack(books_small_emb.values)
films_small_emb = np.stack(films_small_emb.values)

In [50]:
# book_embeddings = pd.concat([
#     books_small[['id', 'title', 'authors']],
#     pd.DataFrame(books_small_emb, columns=[f'emb_book_{i}' for i in range(312)])
# ], axis=1)

# film_embeddings = pd.concat([
#     films_small[['id', 'title', 'year']],
#     pd.DataFrame(films_small_emb, columns=[f'emb_film_{i}' for i in range(312)])
# ], axis=1)

# book_embeddings.to_csv('description_emb_books.csv', index=False)
# film_embeddings.to_csv('description_emb_films.csv', index=False)

In [ ]:
recommendations = []
N = 5


for idx, emb_one_book in zip(list_of_idx_books, books_small_emb):
    sim_coefs = cosine_similarity(emb_one_book.reshape(1, -1), films_small_emb)

    recommender_idxs_by_book = [
        sorted(range(len(sim_coef)), key=lambda sub: sim_coef[sub], reverse=True)[:N]
        for sim_coef in sim_coefs
    ]

    book = books_small[books_small.id == idx]
    films_rec = films_small[films_small.index.isin(recommender_idxs_by_book[0])]

    for film in films_rec.iterrows():
        recommendations.append([
            book.title.tolist()[0],
            book.genres.tolist()[0],
            film[1]['title'],
            film[1]['genres'],
        ])

recommendations = pd.DataFrame(recommendations, columns=['book_title', 'book_genres', 'film_title', 'film_genre'])

    # recommendations[books_small[books_small.id == idx].title.tolist()[0]] = \
    #   films_small[films_small.index.isin(recommender_idxs_by_book[0])].title.tolist()

In [ ]:
recommendations

,book_title,book_genres,film_title,film_genre
0,1984,"Английский язык,Зарубежная фантастика,Социальн...",Все дети Бога могут танцевать,"драмы, зарубежные"
1,1984,"Английский язык,Зарубежная фантастика,Социальн...",Гольфстрим под айсбергом (жестовым языком),"драмы, русские, фэнтези, для взрослых"
2,1984,"Английский язык,Зарубежная фантастика,Социальн...",Холодный март,"драмы, русские"
3,1984,"Английский язык,Зарубежная фантастика,Социальн...",Бродяга Бро,драмы
4,1984,"Английский язык,Зарубежная фантастика,Социальн...",Балаган,"драмы, русские"
5,НИ СЫ. Будь уверен в своих силах и не позволяй...,"Зарубежная психология,Личная эффективность,Мот...",Идеи для жизни,no_genre
6,НИ СЫ. Будь уверен в своих силах и не позволяй...,"Зарубежная психология,Личная эффективность,Мот...",Удача,драмы
7,НИ СЫ. Будь уверен в своих силах и не позволяй...,"Зарубежная психология,Личная эффективность,Мот...",Основы Adobe Illustrator,no_genre
8,НИ СЫ. Будь уверен в своих силах и не позволяй...,"Зарубежная психология,Личная эффективность,Мот...",Йога для начинающих. Основы Йоги,фитнес
9,НИ СЫ. Будь уверен в своих силах и не позволяй...,"Зарубежная психология,Личная эффективность,Мот...",Визуальный язык и его элементы,no_genre


In [ ]:
a = books[books.title == "НИ СЫ. Будь уверен в своих силах и не позволяй сомнениям мешать тебе двигаться вперед"].annotation.tolist()[0]
a

'Мощная антикризисная мотивация. Топ среди книг по саморазвитию в мире\nМышление формирует реальность вокруг нас. Даже в глубоком кризисе можно увидеть новые возможности. Меняйте мышление с помощью «НИ СЫ» и\xa0двигайтесь к новым целям.\nПять причин начать прямо сейчас:\n[ul]С помощью 25 эффективных методик вы станете уверенным в себе и независимым.\nИзбавитесь от страха перед новым делом и будущим.\nОставите обиды и неудачи в прошлом и перестанете искать виноватых.\nПолучите невероятную энергетическую подзарядку, к которой можно возвращаться снова и снова.\nНаучитесь с любовью и пониманием относиться к себе.[/ul]\nДжен Синсеро – самый яркий коуч 21 века, популярный бизнес-тренер и автор мотивационных бестселлеров №1 по версии The New York Times. Читатели особенно ценят ее искрометное чувство юмора и энергетику.\n«Если хочешь заполучить жизнь, которой никогда не жил, придется делать то, что никогда не делал»,\xa0– самое время прислушаться к Джен, не правда ли?'

In [ ]:
b = films[films.title == "Основы Adobe Illustrator"].description.tolist()[0]
b

'Объемный вводный курс, на\xa0котором вы\xa0научитесь пользоваться самым популярным векторным графическим илюстратором.  Пройти полный курс веб-дизайна можно в\xa0GeekBrains, записавшись по\xa0ссылке https://geekbrains.ru/professions/web-designer. Это самый полный курс по\xa0веб-дизайну для тех, кто хочет освоить новую профессию с\xa0нуля, создать мощное портфолио и\xa0пройти стажировку на\xa0реальных проектах Mail.ru Group.'

In [ ]:
set(a.split()).intersection(set(b.split()))

{'в', 'вы', 'и', 'можно', 'по', 'с', 'самый'}

In [ ]:
cosine_similarity(embed_bert_cls(b, model, tokenizer).reshape(1, -1), embed_bert_cls(a, model, tokenizer).reshape(1, -1))

array([[0.7194416]], dtype=float32)

In [ ]:
recommender_idxs_by_book = [
    sorted(range(len(sim_coef)), key=lambda sub: sim_coef[sub], reverse=True)
    for sim_coef in sim_coefs
]

In [ ]:
sorted(sim_coefs[0])[-5:]

[0.82445633, 0.8250097, 0.82556707, 0.8263526, 0.8394168]